# Deep Learning


Objective of this notebook is to check whether the CNN model can be applied on multivariate tabular dataset or not. If yes then improve the performance of the model by setting proper values for hyperparameters and network architecture.
For this purpose, I have taken s&p500 data to predict whether Consumer Price Index will increase or decrease based on the other economic indicators such as __SP500, Dividend, Earnings, Long Interest Rate, Real Price, Real Dividend,  Real Earnings__. 



__Multivariate CNN Models:__
Although traditionally developed for two-dimensional image data, CNNs can be used to model Multivariate time series forecasting problems.

Multivariate time series data means data where there is more than one observation for each time step with a temporal ordering and a model is required to learn from the series of past observations to predict the next value in the sequence.

This section is divided into two parts; they are:

> __A.Data Preparation<br>
> B.CNN Model__<br>

__Data Source: https://datahub.io/core/s-and-p-500__

## A. Data Preparation

In [1]:
import pandas as pd
import numpy as np

In [2]:
sp = pd.read_csv('data_csv.csv')

In [3]:
sp.isnull().values.any()

True

In [4]:
sp.isnull().sum()

Date                      0
SP500                     0
Dividend                  1
Earnings                  4
Consumer Price Index      0
Long Interest Rate        0
Real Price                0
Real Dividend             1
Real Earnings             4
PE10                    120
dtype: int64

In [5]:
sp[sp.isnull().values.any(axis=1)].head()

,Date,SP500,Dividend,Earnings,Consumer Price Index,Long Interest Rate,Real Price,Real Dividend,Real Earnings,PE10
0,1871-01-01,4.44,0.26,0.4,12.46,5.32,89.00,5.21,8.02,NaN
1,1871-02-01,4.50,0.26,0.4,12.84,5.32,87.53,5.06,7.78,NaN
2,1871-03-01,4.61,0.26,0.4,13.03,5.33,88.36,4.98,7.67,NaN
3,1871-04-01,4.74,0.26,0.4,12.56,5.33,94.29,5.17,7.96,NaN
4,1871-05-01,4.86,0.26,0.4,12.27,5.33,98.93,5.29,8.14,NaN


In [6]:
## Droping the column
sp = sp.drop('PE10', axis=1)
## fillna with forward fill
sp=sp.fillna(method='ffill', axis=1)
sp.tail()

,Date,SP500,Dividend,Earnings,Consumer Price Index,Long Interest Rate,Real Price,Real Dividend,Real Earnings
1763,2017-12-01,2664.34,48.93,109.88,246.52,2.4,2700.13,49.59,111.36
1764,2018-01-01,2789.8,49.29,49.29,247.87,2.58,2811.96,49.68,49.68
1765,2018-02-01,2705.16,49.64,49.64,248.99,2.86,2714.34,49.81,49.81
1766,2018-03-01,2702.77,50,50,249.55,2.84,2705.82,50.06,50.06
1767,2018-04-01,2642.19,2642.19,2642.19,249.84,2.8,2642.19,2642.19,2642.19


In [7]:
sp.isnull().sum()

Date                    0
SP500                   0
Dividend                0
Earnings                0
Consumer Price Index    0
Long Interest Rate      0
Real Price              0
Real Dividend           0
Real Earnings           0
dtype: int64

In [12]:
## Use the below code to create a new variable so as to detect increament or decrement of CPI 
'''
cnt=1
target=[]
for i in sp['Consumer Price Index']:
    if sp['Consumer Price Index'][cnt] > i:
        print(sp['Consumer Price Index'][cnt], '>' , i , '1')
        target.append(1)
    else:
        print(sp['Consumer Price Index'][cnt], '<' , i , '0')
        target.append(0)
    cnt+=1
'''

"\ncnt=1\ntarget=[]\nfor i in sp['Consumer Price Index']:\n    if sp['Consumer Price Index'][cnt] > i:\n        print(sp['Consumer Price Index'][cnt], '>' , i , '1')\n        target.append(1)\n    else:\n        print(sp['Consumer Price Index'][cnt], '<' , i , '0')\n        target.append(0)\n    cnt+=1\n"

In [9]:
target=pd.DataFrame(data=target, columns=['CPI_Inc'],dtype=int)
sp=sp[:1767]

In [10]:
frames=[sp,target]
sp=pd.concat(frames, axis=1, join='outer')
sp.tail()

,Date,SP500,Dividend,Earnings,Consumer Price Index,Long Interest Rate,Real Price,Real Dividend,Real Earnings,CPI_Inc
1762,2017-11-01,2593.61,48.68,108.95,246.67,2.35,2626.9,49.3,110.35,0
1763,2017-12-01,2664.34,48.93,109.88,246.52,2.4,2700.13,49.59,111.36,1
1764,2018-01-01,2789.8,49.29,49.29,247.87,2.58,2811.96,49.68,49.68,1
1765,2018-02-01,2705.16,49.64,49.64,248.99,2.86,2714.34,49.81,49.81,1
1766,2018-03-01,2702.77,50,50,249.55,2.84,2705.82,50.06,50.06,1


In [11]:
sp3=sp.drop('Date',axis=1)

test=sp3[-40:]
train=sp3[:-40]
#Train
sp1=np.array(train)
#Test
sp2=np.array(test)

In [14]:
### To change the dimension of the data for deep learning model

from numpy import array

def split_sequences(sequences, n_steps):
    X, y = list(), list()
    for i in range(len(sequences)):
        # find the end of this pattern
        end_ix = i + n_steps
        
        # check if we are beyond the dataset
        if end_ix > len(sequences):
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequences[i:end_ix, :-1], sequences[end_ix-1, -1]
        
        X.append(seq_x)
        y.append(seq_y)
    return array(X), array(y)

In [15]:
# choose a number of time steps
n_steps = 8

# convert into input/output
X1, y1 = split_sequences(sp1, n_steps)
# convert into input/output
X2, y2 = split_sequences(sp2, n_steps)

print(X1.shape, y1.shape)
print(X2.shape, y2.shape)


n_features = X2.shape[2]
n_features

(1720, 8, 8) (1720,)
(33, 8, 8) (33,)


8

# B. CNN Model:

# Part A - Deep Learning model

___Hyperparameters:___<br>
filters=128<br>
kernel_size=3<br>
Activation functions=  relu<br>
loss=mean_squared_error<br>
optimizer= adam<br>

In [16]:
# univariate cnn example
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import BatchNormalization
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras import metrics
from keras import optimizers


Using TensorFlow backend.


In [17]:
# define model
model1 = Sequential()

model1.add(Conv1D(filters=128, kernel_size=3, activation='relu', input_shape=(n_steps, n_features)))
model1.add(MaxPooling1D(pool_size=2))
model1.add(BatchNormalization())

model1.add(Flatten())

model1.add(Dropout(0.25))
model1.add(Dense(20, activation='relu'))
model1.add(Dropout(0.25))
model1.add(Dense(1))


In [18]:
model1.compile(optimizer='adam', loss='mean_squared_error',metrics=[metrics.mae, 'accuracy'])
model1.fit(X1, y1, batch_size=32, epochs=10, verbose=2)

Epoch 1/10
 - 0s - loss: 0.9079 - mean_absolute_error: 0.7118 - accuracy: 0.4372
Epoch 2/10
 - 0s - loss: 0.3815 - mean_absolute_error: 0.5101 - accuracy: 0.5384
Epoch 3/10
 - 0s - loss: 0.2962 - mean_absolute_error: 0.4621 - accuracy: 0.5878
Epoch 4/10
 - 0s - loss: 0.2671 - mean_absolute_error: 0.4447 - accuracy: 0.6122
Epoch 5/10
 - 0s - loss: 0.2616 - mean_absolute_error: 0.4450 - accuracy: 0.6145
Epoch 6/10
 - 0s - loss: 0.2493 - mean_absolute_error: 0.4385 - accuracy: 0.6238
Epoch 7/10
 - 0s - loss: 0.2369 - mean_absolute_error: 0.4315 - accuracy: 0.6401
Epoch 8/10
 - 0s - loss: 0.2305 - mean_absolute_error: 0.4274 - accuracy: 0.6372
Epoch 9/10
 - 0s - loss: 0.2271 - mean_absolute_error: 0.4273 - accuracy: 0.6483
Epoch 10/10
 - 0s - loss: 0.2276 - mean_absolute_error: 0.4295 - accuracy: 0.6488


### Validation:

In [19]:
loss, mean_absolute_error, accuracy =model1.evaluate(X2, y2, batch_size=32, verbose=2)
print('Accuracy on Test Data :- ')
print('\t loss:',loss , '  mean_absolute_error:',mean_absolute_error, '  accuracy:',accuracy)

Accuracy on Test Data :- 
	 loss: 0.6278259826428962   mean_absolute_error: 0.5689218640327454   accuracy: 0.6363636255264282


__For the baseline model, Loss is Higher and Accuracy on validation data is 63%__

# Activation function

___Hyperparameters:___<br>
filters=128<br>
kernel_size=3<br>
Activation functions=  elu, softmax<br>
loss=mean_squared_error<br>
optimizer= adam<br>

In [60]:
# define model
model2 = Sequential()

model2.add(Conv1D(filters=128, kernel_size=3, activation='elu', input_shape=(n_steps, n_features)))
model2.add(MaxPooling1D(pool_size=2))
model2.add(BatchNormalization())

model2.add(Flatten())

model2.add(Dropout(0.25))
model2.add(Dense(20, activation='softmax'))
model2.add(Dropout(0.25))
model2.add(Dense(1))


In [61]:
adam=optimizers.Adam(learning_rate=0.4, beta_1=0.9, beta_2=0.999, amsgrad=False)
model2.compile(optimizer='adam', loss='mean_squared_error',metrics=[metrics.mae, 'accuracy'])
model2.fit(X1, y1, batch_size=32, epochs=10, verbose=2)

Epoch 1/10
 - 0s - loss: 0.3393 - mean_absolute_error: 0.5012 - accuracy: 0.5041
Epoch 2/10
 - 0s - loss: 0.2630 - mean_absolute_error: 0.4662 - accuracy: 0.6250
Epoch 3/10
 - 0s - loss: 0.2531 - mean_absolute_error: 0.4617 - accuracy: 0.6285
Epoch 4/10
 - 0s - loss: 0.2433 - mean_absolute_error: 0.4480 - accuracy: 0.6355
Epoch 5/10
 - 0s - loss: 0.2417 - mean_absolute_error: 0.4452 - accuracy: 0.6273
Epoch 6/10
 - 0s - loss: 0.2390 - mean_absolute_error: 0.4483 - accuracy: 0.6134
Epoch 7/10
 - 0s - loss: 0.2300 - mean_absolute_error: 0.4389 - accuracy: 0.6297
Epoch 8/10
 - 0s - loss: 0.2230 - mean_absolute_error: 0.4265 - accuracy: 0.6424
Epoch 9/10
 - 0s - loss: 0.2205 - mean_absolute_error: 0.4279 - accuracy: 0.6517
Epoch 10/10
 - 0s - loss: 0.2186 - mean_absolute_error: 0.4234 - accuracy: 0.6547


### Validation:

In [63]:
loss, mean_absolute_error, accuracy =model2.evaluate(X2, y2, batch_size=32, verbose=2)
print('Accuracy on Test Data :- ')
print('\t loss:',loss , '  mean_absolute_error:',mean_absolute_error, '  accuracy:',accuracy)

Accuracy on Test Data :- 
	 loss: 0.2764722759073431   mean_absolute_error: 0.523048996925354   accuracy: 0.3030303120613098


__After changing the activation functions in the current network, loss get reduced but with decrease in accuracy on validation set which is 30%.__  

# Cost function / loss

___Hyperparameters:___<br>
filters=128<br>
kernel_size=3<br>
Activation functions=  elu, softmax<br>
loss= hinge, logcosh<br>
optimizer= adam<br>

__1. Cost function= hinge:__

In [23]:
model2.compile(optimizer='adam', loss='hinge',metrics=[metrics.mae, 'accuracy'])
model2.fit(X1, y1, batch_size=32, epochs=10, verbose=2)

Epoch 1/10
 - 0s - loss: 0.8402 - mean_absolute_error: 0.3850 - accuracy: 0.6599
Epoch 2/10
 - 0s - loss: 0.7873 - mean_absolute_error: 0.4537 - accuracy: 0.6541
Epoch 3/10
 - 0s - loss: 0.7639 - mean_absolute_error: 0.5488 - accuracy: 0.5471
Epoch 4/10
 - 0s - loss: 0.7270 - mean_absolute_error: 0.6014 - accuracy: 0.4076
Epoch 5/10
 - 0s - loss: 0.7255 - mean_absolute_error: 0.6384 - accuracy: 0.4070
Epoch 6/10
 - 0s - loss: 0.6938 - mean_absolute_error: 0.7034 - accuracy: 0.3709
Epoch 7/10
 - 0s - loss: 0.7004 - mean_absolute_error: 0.7443 - accuracy: 0.3744
Epoch 8/10
 - 0s - loss: 0.6855 - mean_absolute_error: 0.7608 - accuracy: 0.3744
Epoch 9/10
 - 0s - loss: 0.6869 - mean_absolute_error: 0.7982 - accuracy: 0.3628
Epoch 10/10
 - 0s - loss: 0.6707 - mean_absolute_error: 0.7844 - accuracy: 0.3709


__Accuracy :__

In [24]:
loss, mean_absolute_error, accuracy =model2.evaluate(X2, y2, batch_size=32, verbose=2)
print('Accuracy on Test Data :- ')
print('\t loss:',loss , '  mean_absolute_error:',mean_absolute_error, '  accuracy:',accuracy)

Accuracy on Test Data :- 
	 loss: 0.6813373999162153   mean_absolute_error: 0.3783071041107178   accuracy: 0.6969696879386902


__2. Cost function= logcosh:__

In [25]:
model2.compile(optimizer='adam', loss='logcosh',metrics=[metrics.mae, 'accuracy'])
model2.fit(X1, y1, batch_size=32, epochs=10, verbose=2)

Epoch 1/10
 - 0s - loss: 0.1927 - mean_absolute_error: 0.5205 - accuracy: 0.5320
Epoch 2/10
 - 0s - loss: 0.1088 - mean_absolute_error: 0.4016 - accuracy: 0.6587
Epoch 3/10
 - 0s - loss: 0.1016 - mean_absolute_error: 0.3884 - accuracy: 0.6808
Epoch 4/10
 - 0s - loss: 0.1054 - mean_absolute_error: 0.3997 - accuracy: 0.6669
Epoch 5/10
 - 0s - loss: 0.1062 - mean_absolute_error: 0.4093 - accuracy: 0.6547
Epoch 6/10
 - 0s - loss: 0.1010 - mean_absolute_error: 0.3979 - accuracy: 0.6785
Epoch 7/10
 - 0s - loss: 0.0997 - mean_absolute_error: 0.3970 - accuracy: 0.6860
Epoch 8/10
 - 0s - loss: 0.1005 - mean_absolute_error: 0.4005 - accuracy: 0.6791
Epoch 9/10
 - 0s - loss: 0.1009 - mean_absolute_error: 0.4014 - accuracy: 0.6779
Epoch 10/10
 - 0s - loss: 0.0984 - mean_absolute_error: 0.3983 - accuracy: 0.6837


__Accuracy :__

In [26]:
loss, mean_absolute_error, accuracy =model2.evaluate(X2, y2, batch_size=32, verbose=2)
print('Accuracy on Test Data :- ')
print('\t loss:',loss , '  mean_absolute_error:',mean_absolute_error, '  accuracy:',accuracy)

Accuracy on Test Data :- 
	 loss: 0.10009397159923207   mean_absolute_error: 0.4186834394931793   accuracy: 0.6969696879386902


__For the cost function, logcosh is better than hinge for better accuracy.__ 

# Epochs

___Hyperparameters:___<br>
filters=128<br>
kernel_size=3<br>
Activation functions=  elu, softmax<br>
loss= logcosh<br>
optimizer= adam<br>
epochs=200 <br>

In [27]:
model2.compile(optimizer='adam', loss='logcosh',metrics=[metrics.mae, 'accuracy'])
model2.fit(X1, y1, batch_size=32, epochs=200, verbose=2)

Epoch 1/200
 - 0s - loss: 0.0961 - mean_absolute_error: 0.3912 - accuracy: 0.6971
Epoch 2/200
 - 0s - loss: 0.0988 - mean_absolute_error: 0.3957 - accuracy: 0.6849
Epoch 3/200
 - 0s - loss: 0.0971 - mean_absolute_error: 0.3942 - accuracy: 0.6907
Epoch 4/200
 - 0s - loss: 0.0998 - mean_absolute_error: 0.4024 - accuracy: 0.6837
Epoch 5/200
 - 0s - loss: 0.0964 - mean_absolute_error: 0.3902 - accuracy: 0.6988
Epoch 6/200
 - 0s - loss: 0.0943 - mean_absolute_error: 0.3880 - accuracy: 0.6994
Epoch 7/200
 - 0s - loss: 0.0953 - mean_absolute_error: 0.3900 - accuracy: 0.6977
Epoch 8/200
 - 0s - loss: 0.0964 - mean_absolute_error: 0.3951 - accuracy: 0.6901
Epoch 9/200
 - 0s - loss: 0.0965 - mean_absolute_error: 0.3946 - accuracy: 0.6878
Epoch 10/200
 - 0s - loss: 0.0937 - mean_absolute_error: 0.3886 - accuracy: 0.6971
Epoch 11/200
 - 0s - loss: 0.0939 - mean_absolute_error: 0.3900 - accuracy: 0.7017
Epoch 12/200
 - 0s - loss: 0.0927 - mean_absolute_error: 0.3836 - accuracy: 0.7116
Epoch 13/200


Epoch 100/200
 - 0s - loss: 0.0899 - mean_absolute_error: 0.3746 - accuracy: 0.7244
Epoch 101/200
 - 0s - loss: 0.0906 - mean_absolute_error: 0.3756 - accuracy: 0.7285
Epoch 102/200
 - 0s - loss: 0.0905 - mean_absolute_error: 0.3768 - accuracy: 0.7151
Epoch 103/200
 - 0s - loss: 0.0898 - mean_absolute_error: 0.3793 - accuracy: 0.7203
Epoch 104/200
 - 0s - loss: 0.0892 - mean_absolute_error: 0.3690 - accuracy: 0.7209
Epoch 105/200
 - 0s - loss: 0.0893 - mean_absolute_error: 0.3735 - accuracy: 0.7140
Epoch 106/200
 - 0s - loss: 0.0892 - mean_absolute_error: 0.3731 - accuracy: 0.7262
Epoch 107/200
 - 0s - loss: 0.0908 - mean_absolute_error: 0.3736 - accuracy: 0.7215
Epoch 108/200
 - 0s - loss: 0.0917 - mean_absolute_error: 0.3756 - accuracy: 0.7157
Epoch 109/200
 - 0s - loss: 0.0897 - mean_absolute_error: 0.3762 - accuracy: 0.7203
Epoch 110/200
 - 0s - loss: 0.0892 - mean_absolute_error: 0.3715 - accuracy: 0.7198
Epoch 111/200
 - 0s - loss: 0.0900 - mean_absolute_error: 0.3753 - accuracy:

Epoch 198/200
 - 0s - loss: 0.0893 - mean_absolute_error: 0.3744 - accuracy: 0.7192
Epoch 199/200
 - 0s - loss: 0.0880 - mean_absolute_error: 0.3713 - accuracy: 0.7233
Epoch 200/200
 - 0s - loss: 0.0871 - mean_absolute_error: 0.3631 - accuracy: 0.7209


__Accuracy :__

In [28]:
loss, mean_absolute_error, accuracy =model2.evaluate(X2, y2, batch_size=32, verbose=2)
print('Accuracy on Test Data :- ')
print('\t loss:',loss , '  mean_absolute_error:',mean_absolute_error, '  accuracy:',accuracy)

Accuracy on Test Data :- 
	 loss: 0.11469146699616403   mean_absolute_error: 0.47265517711639404   accuracy: 0.6666666865348816


__After increasing number of epochs in the model, accuracy increased and loss decreased.__

# Gradient estimation

___Hyperparameters:___<br>
filters=128<br>
kernel_size=3<br>
Activation functions=  elu, softmax<br>
loss= logcosh<br>
optimizer= RMSprop, Adagrad<br>
epochs=200 <br>

__1. optimizers : RMSprop__

In [29]:
RMSprop=optimizers.RMSprop(learning_rate=0.4, rho=0.9)
model2.compile(optimizer='RMSprop', loss='logcosh',metrics=[metrics.mae, 'accuracy'])
model2.fit(X1, y1, batch_size=32, epochs=200, verbose=2)

Epoch 1/200
 - 0s - loss: 0.0876 - mean_absolute_error: 0.3621 - accuracy: 0.7169
Epoch 2/200
 - 0s - loss: 0.0890 - mean_absolute_error: 0.3725 - accuracy: 0.7256
Epoch 3/200
 - 0s - loss: 0.0888 - mean_absolute_error: 0.3672 - accuracy: 0.7250
Epoch 4/200
 - 0s - loss: 0.0884 - mean_absolute_error: 0.3677 - accuracy: 0.7198
Epoch 5/200
 - 0s - loss: 0.0874 - mean_absolute_error: 0.3665 - accuracy: 0.7209
Epoch 6/200
 - 0s - loss: 0.0876 - mean_absolute_error: 0.3652 - accuracy: 0.7233
Epoch 7/200
 - 0s - loss: 0.0883 - mean_absolute_error: 0.3668 - accuracy: 0.7308
Epoch 8/200
 - 0s - loss: 0.0869 - mean_absolute_error: 0.3640 - accuracy: 0.7192
Epoch 9/200
 - 0s - loss: 0.0864 - mean_absolute_error: 0.3637 - accuracy: 0.7198
Epoch 10/200
 - 0s - loss: 0.0881 - mean_absolute_error: 0.3665 - accuracy: 0.7238
Epoch 11/200
 - 0s - loss: 0.0868 - mean_absolute_error: 0.3611 - accuracy: 0.7244
Epoch 12/200
 - 0s - loss: 0.0868 - mean_absolute_error: 0.3623 - accuracy: 0.7215
Epoch 13/200


Epoch 100/200
 - 0s - loss: 0.0864 - mean_absolute_error: 0.3591 - accuracy: 0.7366
Epoch 101/200
 - 0s - loss: 0.0869 - mean_absolute_error: 0.3610 - accuracy: 0.7302
Epoch 102/200
 - 0s - loss: 0.0876 - mean_absolute_error: 0.3653 - accuracy: 0.7209
Epoch 103/200
 - 0s - loss: 0.0864 - mean_absolute_error: 0.3603 - accuracy: 0.7134
Epoch 104/200
 - 0s - loss: 0.0866 - mean_absolute_error: 0.3615 - accuracy: 0.7279
Epoch 105/200
 - 0s - loss: 0.0855 - mean_absolute_error: 0.3588 - accuracy: 0.7308
Epoch 106/200
 - 0s - loss: 0.0855 - mean_absolute_error: 0.3589 - accuracy: 0.7349
Epoch 107/200
 - 0s - loss: 0.0871 - mean_absolute_error: 0.3596 - accuracy: 0.7291
Epoch 108/200
 - 0s - loss: 0.0840 - mean_absolute_error: 0.3538 - accuracy: 0.7331
Epoch 109/200
 - 0s - loss: 0.0868 - mean_absolute_error: 0.3582 - accuracy: 0.7221
Epoch 110/200
 - 0s - loss: 0.0859 - mean_absolute_error: 0.3553 - accuracy: 0.7221
Epoch 111/200
 - 0s - loss: 0.0875 - mean_absolute_error: 0.3642 - accuracy:

Epoch 198/200
 - 0s - loss: 0.0862 - mean_absolute_error: 0.3597 - accuracy: 0.7308
Epoch 199/200
 - 0s - loss: 0.0854 - mean_absolute_error: 0.3580 - accuracy: 0.7262
Epoch 200/200
 - 0s - loss: 0.0850 - mean_absolute_error: 0.3553 - accuracy: 0.7395


__Accuracy :__

In [30]:
loss, mean_absolute_error, accuracy =model2.evaluate(X2, y2, batch_size=32, verbose=2)
print('Accuracy on Test Data :- ')
print('\t loss:',loss , '  mean_absolute_error:',mean_absolute_error, '  accuracy:',accuracy)

Accuracy on Test Data :- 
	 loss: 0.10839736822879675   mean_absolute_error: 0.45831698179244995   accuracy: 0.6969696879386902


__2. optimizers : Adagrad__

In [31]:
Adagrad=optimizers.Adagrad(learning_rate=0.001)
model2.compile(optimizer='Adagrad', loss='logcosh',metrics=[metrics.mae, 'accuracy'])
model2.fit(X1, y1, batch_size=32, epochs=200, verbose=2)

Epoch 1/200
 - 0s - loss: 0.0932 - mean_absolute_error: 0.3764 - accuracy: 0.7099
Epoch 2/200
 - 0s - loss: 0.0882 - mean_absolute_error: 0.3672 - accuracy: 0.7227
Epoch 3/200
 - 0s - loss: 0.0882 - mean_absolute_error: 0.3655 - accuracy: 0.7227
Epoch 4/200
 - 0s - loss: 0.0843 - mean_absolute_error: 0.3586 - accuracy: 0.7407
Epoch 5/200
 - 0s - loss: 0.0863 - mean_absolute_error: 0.3626 - accuracy: 0.7279
Epoch 6/200
 - 0s - loss: 0.0870 - mean_absolute_error: 0.3641 - accuracy: 0.7180
Epoch 7/200
 - 0s - loss: 0.0875 - mean_absolute_error: 0.3653 - accuracy: 0.7297
Epoch 8/200
 - 0s - loss: 0.0858 - mean_absolute_error: 0.3620 - accuracy: 0.7302
Epoch 9/200
 - 0s - loss: 0.0849 - mean_absolute_error: 0.3609 - accuracy: 0.7331
Epoch 10/200
 - 0s - loss: 0.0853 - mean_absolute_error: 0.3575 - accuracy: 0.7297
Epoch 11/200
 - 0s - loss: 0.0836 - mean_absolute_error: 0.3537 - accuracy: 0.7413
Epoch 12/200
 - 0s - loss: 0.0846 - mean_absolute_error: 0.3546 - accuracy: 0.7320
Epoch 13/200


Epoch 100/200
 - 0s - loss: 0.0830 - mean_absolute_error: 0.3474 - accuracy: 0.7355
Epoch 101/200
 - 0s - loss: 0.0810 - mean_absolute_error: 0.3455 - accuracy: 0.7407
Epoch 102/200
 - 0s - loss: 0.0816 - mean_absolute_error: 0.3460 - accuracy: 0.7459
Epoch 103/200
 - 0s - loss: 0.0825 - mean_absolute_error: 0.3472 - accuracy: 0.7372
Epoch 104/200
 - 0s - loss: 0.0824 - mean_absolute_error: 0.3451 - accuracy: 0.7401
Epoch 105/200
 - 0s - loss: 0.0810 - mean_absolute_error: 0.3455 - accuracy: 0.7506
Epoch 106/200
 - 0s - loss: 0.0810 - mean_absolute_error: 0.3446 - accuracy: 0.7494
Epoch 107/200
 - 0s - loss: 0.0827 - mean_absolute_error: 0.3485 - accuracy: 0.7378
Epoch 108/200
 - 0s - loss: 0.0820 - mean_absolute_error: 0.3459 - accuracy: 0.7488
Epoch 109/200
 - 0s - loss: 0.0811 - mean_absolute_error: 0.3443 - accuracy: 0.7465
Epoch 110/200
 - 0s - loss: 0.0838 - mean_absolute_error: 0.3487 - accuracy: 0.7355
Epoch 111/200
 - 0s - loss: 0.0817 - mean_absolute_error: 0.3471 - accuracy:

Epoch 198/200
 - 0s - loss: 0.0826 - mean_absolute_error: 0.3470 - accuracy: 0.7384
Epoch 199/200
 - 0s - loss: 0.0807 - mean_absolute_error: 0.3456 - accuracy: 0.7576
Epoch 200/200
 - 0s - loss: 0.0819 - mean_absolute_error: 0.3453 - accuracy: 0.7500


In [32]:
loss, mean_absolute_error, accuracy =model2.evaluate(X2, y2, batch_size=32, verbose=2)
print('Accuracy on Test Data :- ')
print('\t loss:',loss , '  mean_absolute_error:',mean_absolute_error, '  accuracy:',accuracy)

Accuracy on Test Data :- 
	 loss: 0.10720591111616655   mean_absolute_error: 0.462567538022995   accuracy: 0.6969696879386902


__Optimizer: Adagrad is better suit for this model than RMSprop. Although, both are close enough in terms of accuracy and loss.__ 

# Network Architecture

___Hyperparameters:___<br>
filters=256<br>
kernel_size=2<br>
Activation functions=  relu, elu, softmax<br>
loss=logcosh<br>
optimizer= adagrad<br>
epoch=100<br>

In [55]:
# define model
model3 = Sequential()

model3.add(Conv1D(filters=256, kernel_size=3, activation='relu', input_shape=(n_steps, n_features)))
model3.add(MaxPooling1D(pool_size=2))
model3.add(BatchNormalization())

model3.add(Dense(128, activation='elu'))
model3.add(Dropout(0.25))

model3.add(Conv1D(filters=128, kernel_size=2, activation='elu'))
model3.add(MaxPooling1D(pool_size=2))
model3.add(BatchNormalization())
model3.add(Dropout(0.25))

model3.add(Dense(64))
model3.add(Dropout(0.25))


model3.add(Flatten())

model3.add(Dropout(0.25))
model3.add(Dense(20, activation='softmax'))
model3.add(Dropout(0.25))
model3.add(Dense(1))


In [56]:
Adagrad=optimizers.Adagrad(learning_rate=0.4)
model3.compile(optimizer='Adagrad', loss='logcosh',metrics=[metrics.mae, 'accuracy'])
model3.fit(X1, y1, batch_size=32, epochs=100, verbose=2)

Epoch 1/100
 - 1s - loss: 0.1323 - mean_absolute_error: 0.4539 - accuracy: 0.6233
Epoch 2/100
 - 0s - loss: 0.1209 - mean_absolute_error: 0.4423 - accuracy: 0.6233
Epoch 3/100
 - 0s - loss: 0.1179 - mean_absolute_error: 0.4373 - accuracy: 0.6203
Epoch 4/100
 - 0s - loss: 0.1130 - mean_absolute_error: 0.4227 - accuracy: 0.6459
Epoch 5/100
 - 0s - loss: 0.1164 - mean_absolute_error: 0.4331 - accuracy: 0.6238
Epoch 6/100
 - 0s - loss: 0.1142 - mean_absolute_error: 0.4335 - accuracy: 0.6349
Epoch 7/100
 - 0s - loss: 0.1151 - mean_absolute_error: 0.4348 - accuracy: 0.6267
Epoch 8/100
 - 0s - loss: 0.1150 - mean_absolute_error: 0.4368 - accuracy: 0.6267
Epoch 9/100
 - 0s - loss: 0.1118 - mean_absolute_error: 0.4339 - accuracy: 0.6302
Epoch 10/100
 - 0s - loss: 0.1065 - mean_absolute_error: 0.4257 - accuracy: 0.6564
Epoch 11/100
 - 0s - loss: 0.1123 - mean_absolute_error: 0.4346 - accuracy: 0.6145
Epoch 12/100
 - 0s - loss: 0.1087 - mean_absolute_error: 0.4308 - accuracy: 0.6360
Epoch 13/100


Epoch 100/100
 - 0s - loss: 0.0940 - mean_absolute_error: 0.3915 - accuracy: 0.6965


In [59]:
loss, mean_absolute_error, accuracy =model3.evaluate(X2, y2, batch_size=32, verbose=2)
print('Accuracy on Test Data :- ')
print('\t loss:',loss , '  mean_absolute_error:',mean_absolute_error, '  accuracy:',accuracy)

Accuracy on Test Data :- 
	 loss: 0.09973003647544167   mean_absolute_error: 0.4202899932861328   accuracy: 0.6969696879386902


__After changing the number of layers and the size of the layers, both accuracy and loss got improve.__

# Network initialization

___Hyperparameters:___<br>
filters=256<br>
kernel_size=3,2<br>
Activation functions= relu, elu, softmax<br>
kernel_initializer=Ones, RandomNormal <br>
loss=mean_squared_error<br>
optimizer= adam<br>

In [35]:
# define model
# Default : kernel_initializer='glorot_uniform'

model4 = Sequential()

model4.add(Conv1D(filters=256, kernel_size=3, activation='relu', kernel_initializer='Ones' , input_shape=(n_steps, n_features)))
model4.add(MaxPooling1D(pool_size=2))
model4.add(BatchNormalization())


model4.add(Conv1D(filters=128, kernel_size=2, activation='elu',kernel_initializer='RandomNormal'))
model4.add(MaxPooling1D(pool_size=2))
model4.add(BatchNormalization())
model4.add(Dropout(0.25))


model4.add(Flatten())

model4.add(Dropout(0.25))
model4.add(Dense(20, activation='softmax'))
model4.add(Dropout(0.25))
model4.add(Dense(1))


In [36]:
Adagrad=optimizers.Adagrad(learning_rate=0.01)
model4.compile(optimizer='Adagrad', loss='logcosh',metrics=[metrics.mae, 'accuracy'])
model4.fit(X1, y1, batch_size=32, epochs=200, verbose=2)

Epoch 1/100
 - 0s - loss: 0.1557 - mean_absolute_error: 0.5087 - accuracy: 0.5331
Epoch 2/100
 - 0s - loss: 0.1251 - mean_absolute_error: 0.4715 - accuracy: 0.6116
Epoch 3/100
 - 0s - loss: 0.1203 - mean_absolute_error: 0.4564 - accuracy: 0.6134
Epoch 4/100
 - 0s - loss: 0.1118 - mean_absolute_error: 0.4376 - accuracy: 0.6442
Epoch 5/100
 - 0s - loss: 0.1156 - mean_absolute_error: 0.4476 - accuracy: 0.6169
Epoch 6/100
 - 0s - loss: 0.1150 - mean_absolute_error: 0.4465 - accuracy: 0.6227
Epoch 7/100
 - 0s - loss: 0.1136 - mean_absolute_error: 0.4402 - accuracy: 0.6279
Epoch 8/100
 - 0s - loss: 0.1117 - mean_absolute_error: 0.4401 - accuracy: 0.6337
Epoch 9/100
 - 0s - loss: 0.1107 - mean_absolute_error: 0.4356 - accuracy: 0.6430
Epoch 10/100
 - 0s - loss: 0.1111 - mean_absolute_error: 0.4376 - accuracy: 0.6308
Epoch 11/100
 - 0s - loss: 0.1123 - mean_absolute_error: 0.4390 - accuracy: 0.6320
Epoch 12/100
 - 0s - loss: 0.1110 - mean_absolute_error: 0.4356 - accuracy: 0.6308
Epoch 13/100


Epoch 100/100
 - 0s - loss: 0.1028 - mean_absolute_error: 0.4246 - accuracy: 0.6564


In [37]:
loss, mean_absolute_error, accuracy =model4.evaluate(X2, y2, batch_size=32, verbose=2)
print('Accuracy on Test Data :- ')
print('\t loss:',loss , '  mean_absolute_error:',mean_absolute_error, '  accuracy:',accuracy)

Accuracy on Test Data :- 
	 loss: 0.09965821287848732   mean_absolute_error: 0.41854360699653625   accuracy: 0.6969696879386902


__After changing the kernel initialization, accuracy on validation set got increase.__

# Conclusion:
After performing analysis on the CNN model, it is observed that, <br>
>1. For the baseline model, Loss is Higher and Accuracy on validation data is 63%.<br> 
>2. The cost function logcosh is appropriate for this model than hinge.<br> 
>3. Higher the number of epochs, better the model is.<br>
>4. Change in Optimizer can increase the accuracy and it helps to reduce the loss. Adagrad is better suit for this type of model than RMSprop<br>
>5. Network architecture is as important as hyperparameters. By adding more layers and with proper use of hyperparamters, we can achieve higher accuracy.<br>
>6. After changing the kernel initialization, accuracy on validation set got increase.

For Multivariate Tabular data, Convolution Neural Network can produce good results but handling of data For the CNN model is difficult. 

# Author: 
Pranav Khiste( NUID : 001057866 )<br>
Information Systems <br>
Northeastern University <br>

# Citation:
References: <br>
https://keras.io/models<br>
https://machinelearningmastery.com/ <br>
https://karpathy.github.io/2019/04/25/recipe<br>

# Licensing

Copyright 2020 Pranav Sanjay Khiste

Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.